In [1]:
# 加载pth文件
import torch
import json
import os
import decord
import cv2
from tqdm import tqdm

In [17]:
txt = torch.load("txt_embedv2/txt2_eval_tv2v.pth")

In [18]:
txt.keys()

dict_keys(['texts', 'feats'])

In [19]:
len(txt['texts'])

1000

In [11]:
txt['feats'].shape

torch.Size([1000, 256])

In [2]:
with open("/share/huaying/long_video/video_retriever/benchmark/retrieval/webvid/eval_tv2v.jsonl") as f:
    data = json.load(f)

In [3]:
tgt_video = [item["tgt_video_path"][0].split("/")[-1] for item in data]

In [4]:
tgt_video_set = set(tgt_video)

In [5]:
def encode_set(input_set):
    encoded_list = dict()
    for index, value in enumerate(input_set):
        value = value.split('/')[-1]
        encoded_list[value] = index
    return encoded_list


# 调用函数并打印结果
# encoded_tgt_video_set = encode_set(tgt_video_set)


In [7]:
# qry_video_path
for item in tqdm(data):
    video_path = item['qry_video_path']
    # 保存最中间的一帧
    # 使用 OpenCV 打开视频文件
    cap = cv2.VideoCapture(video_path)
    
    # 获取视频的总帧数
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # 计算中间帧的索引
    middle_frame_index = total_frames // 2
    
    # 设置视频读取的帧数为中间帧的索引
    cap.set(cv2.CAP_PROP_POS_FRAMES, middle_frame_index)
    
    # 读取中间帧
    ret, frame = cap.read()
    
    # 关闭视频文件
    cap.release()
    
    # 保存中间帧为图片文件
    if ret:
        cv2.imwrite(f'qry_imgsv2/{item["qry_video_path"].split("/")[-1]}.jpg', frame)
    else:
        print(f"Failed to read frame from {video_path}")
    

100%|██████████| 1000/1000 [00:44<00:00, 22.34it/s]


In [8]:
candidate_pth_list = os.listdir("/share/liangzy/CoVR_code/embedv2")

In [9]:
for i in range(len(candidate_pth_list)):
    candidate_pth_list[i] = f"/share/liangzy/CoVR_code/embedv2/{candidate_pth_list[i]}"

In [10]:
candidate_pth_list[0]

'/share/liangzy/CoVR_code/embedv2/1043906161.mp4.pth'

In [11]:
temp = torch.load(candidate_pth_list[0])

In [12]:
temp.shape

torch.Size([15, 256])

In [13]:
candidate = []
for item in candidate_pth_list:
    candidate.append(torch.load(item))

In [14]:
encoded_tgt_video_set = encode_set(candidate_pth_list)

In [ ]:
data[100]

In [15]:
gt_truth = []
for item in data:
    q = item['tgt_video_path'][0].split('/')[-1]
    s = f"{q}.pth"
    idx = encoded_tgt_video_set[s]
    gt_truth.append(idx)

In [20]:
qry = txt['feats']

In [21]:
qry.shape

torch.Size([1000, 256])

In [22]:
cad = torch.stack(candidate)

In [23]:
cad.shape

torch.Size([977, 15, 256])

In [24]:
sim = torch.einsum('ad,bfd->abf', qry, cad)

In [25]:
sim.shape

torch.Size([1000, 977, 15])

In [26]:
sim_sum = sim.sum(dim=-1)

In [27]:
sim_sum.shape

torch.Size([1000, 977])

In [28]:
# 选出最大的
_, max_idx = torch.max(sim_sum, dim=1)

In [29]:
gt_truth_tensor = torch.tensor(gt_truth)

In [30]:
# 对比两个张量
equal_elements = max_idx == gt_truth_tensor  # 返回一个布尔张量，表示每个位置是否相等

# 统计相同的数量
equal_count = equal_elements.sum().item()  # 将布尔张量的 True 转为 1，并统计总数

In [31]:
equal_count

488

In [32]:
equal_count / 1000

0.488